In [32]:
from pygame.locals import *
import pygame
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

def cube():
    vertices = (
        (1,-1,-1), (1,1,-1), (-1,1,-1), (-1,-1,-1),
        (1,-1,1), (1,1,1), (-1,-1,1), (-1,1,1)
    )
    edegs = (
        (0,1), (0,3), (0,4), (2,1), (2,3), (2,7),
        (6,3), (6,4), (6,7), (5,1), (5,4), (5,7),
    )
    glBegin(GL_LINES)
    glColor3fv((1,1,1))
    for edeg in edegs:
        for vertex in edeg:
            glVertex3fv(vertices[vertex])
    glColor3fv((1,0,0))    
    glVertex3f(0, -2, 0)  
    glVertex3f(0, 2, 0)    
    glEnd()

pygame.init()
screen = (400, 300)
pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)

glEnable(GL_DEPTH_TEST)

glMatrixMode(GL_PROJECTION)
gluPerspective(45, (screen[0]/screen[1]), 0.1, 50.0)

glMatrixMode(GL_MODELVIEW)  
glTranslate(0.0,0.0,-5)

rot_x, rot_y, zoom = 0, 0, -0.5

clock = pygame.time.Clock()
busy = True
while busy:

    mouse_buttons = pygame.mouse.get_pressed()
    button_down = mouse_buttons[0] == 1
   
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            busy = False
        elif event.type == pygame.MOUSEMOTION:
            if button_down:
                rot_x += event.rel[1]
                rot_y += event.rel[0]
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 4:
                zoom += 0.2
            if event.button == 5:
                zoom -= 0.2
            
    glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)

    glPushMatrix()
    glTranslatef(0.0,0.0, zoom)
    glRotatef(rot_x, 1, 0, 0)    
    glRotatef(rot_y, 0, 1, 0)    
    cube()
    glPopMatrix()
    
    pygame.display.flip()
    clock.tick(100)

pygame.quit()